# **INTELIGENCIA ARTIFICIAL APLICADA A LA CIBERSEGURIDAD**
## **PRÁCTICA P3 - BLOQUE III**

**INSTRUCCIONES / RECOMENDACIONES**

- Se recomienda leer con detalle la descripción de cada una de las celdas.
- Las celdas que ya tienen código, se deberán ejecutar directamente. NOTA: algunas celdas con código deberán parametrizarse según lo indicado en cada caso.
- Las celdas que están vacías, se completarán con la implementación requerida en el notebook.
- No se incluirán más celdas de las establecidas en el presente notebook, por lo que la solución al mismo deberá implementarse exclusivamente en las celdas vacías.
- La entrega se realizará vía Moodle. Será necesario subir la solución a este notebook con el nombre: **NOMBRE_GRUPO.ipynb**

- **Fecha de Publicación: 08/04/2024**
- **Fecha de Entrega: 14/04/2024**
- **Test: 15/04/2024**

In [1]:
# SETUP

!pip3 install transformers
!pip3 install einops
!pip3 install accelerate
!pip3 install unstructured-pytesseract
!pip3 install unstructured-inference
!pip3 install sentence_transformers
!pip3 install chromadb

#!pip3 install protobuf==3.20.*

!pip3 install langchain

In [2]:
!pip3 install unstructured
!pip3 install pillow_heif
#!pip3 install cmake
!pip3 install pikepdf pypdf
#!pip3 install python-poppler

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 16.3 MB/s eta 0:00:00


In [3]:
# IMPORTS

from transformers import AutoTokenizer
import transformers
import torch

import langchain

langchain.__version__

'0.1.16'

## LLM

Se deberá parametrizar el modelo LLM según los siguientes atributos:
- model_id
- max_length
- max_new_tokens

In [4]:
# LLM using HuggingFace GPT2

from langchain import HuggingFacePipeline

llm = HuggingFacePipeline.from_model_id(
    model_id="gpt2",
    task="text-generation",
    model_kwargs={
        "max_length": 1024,
        'do_sample': True,
        'top_k': 10,
        'num_return_sequences': 2,
        #'device_map': 'auto',
        'trust_remote_code': True,
        'torch_dtype': torch.bfloat16
    },
    pipeline_kwargs={"max_new_tokens": 50},
    device=0, # With GPU
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

## DOCUMENTS & SPLITTER & VECTORSTORE

Se deberá construir un sistema RAG sobre un caso de uso de Ciberseguridad, utilizando para ello el modelo LLM especificado. La selección del caso de uso de Ciberseguridad será propuesta por el alumn@, estableciendo el documento PDF base (**pdf_base**) para realizar posteriormente las consultas al modelo.

In [5]:
# INDEXING

from langchain.document_loaders import OnlinePDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import ConversationalRetrievalChain

In [6]:
# 1. LOAD
loader = OnlinePDFLoader("https://dn720004.ca.archive.org/0/items/bloodborne-collectors-edition-guide/BLOODBORNE%20Collector%E2%80%99s%20Edition%20Guide_text.pdf")
document = loader.load()

# 2. SPLIT
text_splitter = CharacterTextSplitter(chunk_size=512, chunk_overlap=64)
documents = text_splitter.split_documents(document)

# 3. EMBED & STORE
embeddings = HuggingFaceEmbeddings()
vectorstore = Chroma.from_documents(documents, embeddings)

# RETRIEVAL & GENERATION
qa = ConversationalRetrievalChain.from_llm(
    llm,
    vectorstore.as_retriever(),
    return_source_documents=True,
)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## QUESTION TO LLM = MY_QUERY + CONTEXT_from_VECTORSTORE -> ANSWER FROM LLM

Se deberá generar un catálogo de 10 prompts de consultas sobre el sistema RAG generado, utilizando para ello la variable **my_query**. Los prompts generados deberán obtener respuestas del modelo LLM con una calidad adecuada.

In [7]:
# QUESTION TO LLM

import warnings
warnings.filterwarnings('ignore')

chat_history = []

In [8]:
my_query = "Where to obtain ludwig's holy sword?"
chat_history = []

In [9]:
result = qa({"question": my_query, "chat_history": chat_history})
print(result["answer"])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

You'll find this — on a corpse in the Orphanage of the Msc Ca- thedral Ward p142. Although you won't be able to reach it until very late, picking it up allows you to purchase Twin Bloodstone Shards from the shop. This will allow you to experiment with a wider variety of weapons. The Sedatives it unlocks are also useful, as any hunter who has stared down a Winter Lantern should know!

of Ludwig. These hunters, also known as Holy

—

e Á ee mpm

A trick weapon typically used by Healing Church hunters. It is said that the silver sword

was employed by Ludwig, the first hunter of the church. When transformed, it com-

bines with its sheath to form a greatsword. The Healing Church workshop began with

Ludwig, and departed from old Gehrman's techniques to provide hunters with the

means to hunt more terrifying beasts, and perhaps 

In [10]:
my_query = "What are the starting weapons?"
chat_history = []

In [11]:
result = qa({"question": my_query, "chat_history": chat_history})
print(result["answer"])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Bring the Cannon anda single-shot firearm like the Hunter Pistol or Evelyn, and be sure to purchase a full load of Bone Marrow Ash and Quicksilver Bullets; the Repeating Pistol is optional for higher burst damage, but consumes ammunition twice as quickly. Throwing Knives and Molotovs are useful for backup ranged damage if you run out of bullets, and Oil Urns are handy if you plan on using Mol- otovs or the Flamesprayer; Fire and Bolt Paper are equally effective for increasing the power of your melee attacks.

Undead Giant (Hatchet & Cannon} lefivuimmtie abyrin y :

arriors : Labyrinth Warrior (Greatsword) Labyrinth Warrior (Crossbow & Sword)

Labyrinth Warrior (Morning Star) Labyrinth Warrior (Sword & Shield)

Loran Cleric neenecs Hunting Dog Hunter Enemies VENTE ahar gul (Rifle Spear & Ludwig's Holy Rifle) Yahar'gul Hunter 

# Conclusiones

Detalle las conclusiones extraídas del presente estudio sobre LLMs y sistemas RAG.

*# Escriba aquí las conclusiones*